In [ ]:
%env SM_FRAMEWORK=tf.keras

import segmentation_models as sm

from segmentation_models.utils import set_trainable
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.legacy import Adam, SGD

import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans
import matplotlib.pyplot as plt
import cv2
import time
from keras.callbacks import ModelCheckpoint, CSVLogger

In [ ]:
def adjust_data(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask >= 0.5] = 1
        mask[mask < 0.5] = 0
    return (img,mask)



In [ ]:
def train_generator(batch_size, 
                   train_path,
                   image_folder,
                   mask_folder,
                   aug_dict,
                   image_color_mode = "grayscale",
                   mask_color_mode = "grayscale", 
                   image_save_prefix  = "image", 
                   mask_save_prefix  = "mask",
                   flag_multi_class = False,
                   num_class = 1,
                   save_to_dir = None,
                   target_size = (width_size,height_size),
                   seed = 1):
    '''
    Can generate image and mask at the same time. Use the same seed for image_datagen and mask_datagen to ensure 
    the transformation for image and mask is the same. If you want to visualize the results of generator, set 
    save_to_dir = "your path"
    '''
    
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    
    image_generator = image_datagen.flow_from_directory(train_path,
                                                        classes = [image_folder],
                                                        class_mode = None,
                                                        color_mode = image_color_mode,
                                                        target_size = target_size,
                                                        batch_size = batch_size,
                                                        save_to_dir = save_to_dir,
                                                        save_prefix  = image_save_prefix,
                                                        seed = seed)
    
    mask_generator = mask_datagen.flow_from_directory(train_path,
                                                        classes = [mask_folder],
                                                        class_mode = None,
                                                        color_mode = mask_color_mode,
                                                        target_size = target_size,
                                                        batch_size = batch_size,
                                                        save_to_dir = None,
                                                        save_prefix  = mask_save_prefix,
                                                        seed = seed)
    
    train_generator = zip(image_generator, mask_generator)
    
    for (img,mask) in train_generator:
        img,mask = adjust_data(img, mask, flag_multi_class, num_class)
        yield (img,mask)

In [ ]:
def test_generator(batch_size,
                  test_path,
                  image_folder,
                  mask_folder,
                  image_color_mode = "grayscale",
                  mask_color_mode = "grayscale",
                  image_save_prefix  = "image",
                  mask_save_prefix  = "mask",
                  flag_multi_class = False,
                  num_class = 1, 
                  save_to_dir = None,
                  target_size = (width_size,height_size), seed = 2):
    '''
    Can generate image and mask at the same time. Use the same seed for image_datagen and mask_datagen to ensure 
    the transformation for image and mask is the same. If you want to visualize the results of generator, set 
    save_to_dir = "your path"
    '''
    
    image_datagen = ImageDataGenerator()
    mask_datagen = ImageDataGenerator()
    
    image_generator = image_datagen.flow_from_directory(test_path,
                                                        classes = [image_folder],
                                                        class_mode = None,
                                                        color_mode = image_color_mode,
                                                        target_size = target_size,
                                                        batch_size = batch_size,
                                                        save_to_dir = save_to_dir,
                                                        save_prefix  = image_save_prefix,
                                                        seed = seed)
    
    mask_generator = mask_datagen.flow_from_directory(test_path,
                                                      classes = [mask_folder],
                                                      class_mode = None,
                                                      color_mode = mask_color_mode,
                                                      target_size = target_size,
                                                      batch_size = batch_size,
                                                      save_to_dir = save_to_dir,
                                                      save_prefix  = mask_save_prefix,
                                                      seed = seed)
    
    test_generator = zip(image_generator, mask_generator)
    
    for (img,mask) in test_generator:
        img,mask = adjust_data(img,mask,flag_multi_class,num_class)
        yield (img,mask)

In [ ]:
path_dataset = '../dataset_insulators/'
train_data_path = '../dataset_insulators/train/images'
test_data_path = '../dataset_insulators/validation/images/'

num_of_imgs_train = sum(len(files) for _, _, files in os.walk(train_data_path))
num_of_imgs_test = sum(len(files) for _, _, files in os.walk(test_data_path))

num_of_train_samples = int(num_of_imgs_train)
num_of_validation_samples  = int(num_of_imgs_test)

print("Train Images : ", num_of_train_samples)
print("Validation Images  : ", num_of_validation_samples)

batch_size = 2
train_samples = num_of_train_samples
test_samples = num_of_validation_samples
steps_p_epoch = int(train_samples / batch_size)
val_steps = int(test_samples / batch_size)
nr_epochs = 250

In [ ]:
model_Unet_resnet34 = sm.Unet(backbone_name='resnet34', encoder_weights='imagenet', encoder_freeze=True, classes=1, activation='sigmoid')
model_Linknet_resnet34 = sm.Linknet(backbone_name='resnet34', encoder_weights='imagenet', encoder_freeze=True, classes=1, activation='sigmoid')
model_Unet_vgg16 = sm.Unet(backbone_name='vgg16', encoder_weights='imagenet', encoder_freeze=True, classes=1, activation='sigmoid')
model_Linknet_vgg16 = sm.Linknet(backbone_name='vgg16', encoder_weights='imagenet', encoder_freeze=True, classes=1, activation='sigmoid')

all_models = [model_Unet_resnet34, model_Linknet_resnet34, model_Unet_vgg16, model_Linknet_vgg16]
all_names_models = ["model_Unet_resnet34", "model_Linknet_resnet34", "model_Unet_vgg16", "model_Linknet_vgg16"]

In [ ]:
for index in range(len(all_names_models)):
    since = time.time()
    BACKBONE = all_names_models[index].split('_')[-1]
    DECODER = all_names_models[index].split('_')[-2]
    
    preprocess_input = sm.get_preprocessing(BACKBONE)

    trainGene = train_generator(batch_size, 
                               path_dataset, 
                               'train',
                               'mask_train',
                               data_gen_args,
                               save_to_dir = None, 
                               image_color_mode = "rgb"
                              )

    testGene = test_generator(batch_size, 
                              path_dataset,
                              'validation', 
                              'mask_validation', 
                              save_to_dir = None,
                              image_color_mode = "rgb")
    
    model = all_models[index]
    opt = Adam(learning_rate=1e-3)

    model.compile(opt,
                  loss=sm.losses.jaccard_loss,
                  metrics=[sm.metrics.IOUScore(threshold=0.5),
                  sm.metrics.FScore(threshold=0.5), 
                  sm.metrics.precision])

    csv_logger = CSVLogger(BACKBONE + "_" + DECODER + ".csv", append=True)
    
    model_checkpoint = ModelCheckpoint(BACKBONE + "_" + DECODER + ".hdf5", 
                                       monitor='loss',
                                       verbose=1,
                                       save_best_only=True)
    
    history = model.fit(trainGene, 
                        validation_data = testGene,
                        validation_steps = val_steps,
                        steps_per_epoch = steps_p_epoch,
                        epochs=nr_epochs,
                        callbacks=[model_checkpoint, csv_logger])
    
    time_elapsed = time.time() - since
    print(
        f"\n Total time: {round(time_elapsed // 60, 2)} minutes and {round(time_elapsed % 60, 2)} seconds"
    )
